In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import env

In [7]:
def get_zillow_2017():   
    if os.path.exists('zillow_2017.csv'):
        return pd.read_csv('zillow_2017.csv',index_col=0)
    else:
        url = env.get_connection('zillow')
        query = 'select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, \
        taxamount, fips  from properties_2017 join propertylandusetype using (propertylandusetypeid) where \
        propertylandusedesc = "Single Family Residential"'
        df = pd.read_sql(query,url)
        df.to_csv('zillow_2017.csv')
        return df

In [8]:
df = get_zillow_2017()

In [11]:
# 2.1 million rows, 7 columns
df.shape

(2152863, 7)

In [13]:
# FIPS (a unique county identifier code), all columns should be numeric
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


taxvaluedollarcnt: total tax assessed value of the parcel  
taxamount: total property tax assessed yearly

In [15]:
# All dtypes are numeric - that's good
df.dtypes

bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                       float64
taxamount                       float64
fips                            float64
dtype: object

In [20]:
# There are 2152863 rows - 9337 nan values is less than .5% of the data
df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [27]:
# If we drop all nan values we drop from 2152863 to 2140235 = keeping 99.4% of data - good enough!
df.dropna().shape

(2140235, 7)

In [29]:
df = df.dropna()

In [35]:
#Looking for errors in data - 18 bedrooms and 0 bathrooms?
df[df.bedroomcnt>15]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
26485,25.0,20.0,11700.0,1608491.0,2010.0,19238.87,6037.0
650428,16.0,5.0,2584.0,543654.0,1922.0,6490.08,6037.0
799516,18.0,18.0,9309.0,2325000.0,1986.0,29283.87,6037.0
1661135,18.0,0.0,26116.0,20260690.0,2002.0,243003.64,6037.0
1758193,18.0,18.0,11134.0,2353308.0,1986.0,29165.39,6037.0
1866663,16.0,16.0,7121.0,975000.0,1987.0,11218.15,6037.0


In [37]:
# 5326 rows have either 0 bedrooms or bathrooms - again, such a small amount that it's worth dropping
# These are single family residential, so not looking at land without structures or a barn
df[(df.bedroomcnt==0) | (df.bathroomcnt==0)]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
15,0.0,0.0,203.0,14166.0,1960.0,163.79,6037.0
398,3.0,0.0,2239.0,396608.0,1988.0,4404.22,6111.0
753,3.0,0.0,2421.0,1111120.0,1976.0,12394.16,6111.0
...,...,...,...,...,...,...,...
2149054,0.0,0.0,938.0,56733.0,1978.0,986.86,6037.0
2151453,2.0,0.0,1108.0,937075.0,1932.0,10851.50,6059.0
2152505,0.0,0.0,240.0,31234.0,1942.0,414.04,6037.0
2152704,0.0,1.0,1490.0,152000.0,1930.0,3614.40,6037.0


In [40]:
# Drop all rows with either no bedrooms or bathrooms
df = df.drop(df[(df.bedroomcnt==0) | (df.bathroomcnt==0)].index)

In [44]:
# Drop all properties under 250 square feet (this is a small tiny home)
df = df.drop(df[df.calculatedfinishedsquarefeet<250].index)

In [46]:
# Drop all properties where value is less than taxes
df = df.drop(df[df.taxvaluedollarcnt<df.taxamount].index)

In [49]:
# Drop all properties built before 1850 - suspicious
df = df.drop(df[df.yearbuilt<1850].index)

In [52]:
# Drop properties that have more bathrooms than bedrooms
df = df.drop(df[df.bedroomcnt<df.bathroomcnt].index)

In [53]:
df.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.054416e+06,2.054416e+06,2.054416e+06,2.054416e+06,2.054416e+06,2.054416e+06,2.054416e+06
mean,3.300318e+00,2.154704e+00,1.784704e+03,4.114300e+05,1.960196e+03,5.054941e+03,6.048199e+03
std,9.028749e-01,8.356171e-01,1.028605e+03,4.400658e+05,2.161895e+01,5.083658e+03,2.034288e+01
min,1.000000e+00,5.000000e-01,2.500000e+02,7.220000e+02,1.855000e+03,1.648000e+01,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.248000e+03,1.850130e+05,1.949000e+03,2.494320e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.600000e+03,3.192950e+05,1.958000e+03,4.013060e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.138000e+03,5.101502e+05,1.974000e+03,6.144185e+03,6.059000e+03
max,2.500000e+01,2.000000e+01,9.525760e+05,5.025488e+07,2.016000e+03,6.006166e+05,6.111000e+03


In [56]:
df.bedroomcnt.min()

1.0

In [58]:
def wrangle_zillow():
    df = get_zillow_2017()

    # Lose very small amount dropping nan values
    df = df.dropna()
    # Drop all rows with either no bedrooms or bathrooms
    df = df.drop(df[(df.bedroomcnt==0) | (df.bathroomcnt==0)].index)
    # Drop all properties under 250 square feet (this is a small tiny home)
    df = df.drop(df[df.calculatedfinishedsquarefeet<250].index)
    # Drop all properties where value is less than taxes
    df = df.drop(df[df.taxvaluedollarcnt<df.taxamount].index)
    # Drop all properties built before 1850 - suspicious
    df = df.drop(df[df.yearbuilt<1850].index)
    # Drop properties that have more bathrooms than bedrooms
    df = df.drop(df[df.bedroomcnt<df.bathroomcnt].index)
    return df

# Based on the work you've done, choose a scaling method for your dataset. Write a function within your prepare.py that accepts as input the train, validate, and test data splits, and returns the scaled versions of each. Be sure to only learn the parameters for scaling from your training data!